# Ingestão Bronze - Controle Ativo

Este notebook realiza a leitura de um arquivo CSV contendo o controle de Ações e FII
e grava os dados em formato Delta na camada **Bronze**.

**Origem dos dados:**
- Arquivo: `D:\Projetos\Jornada_financas_pessoais\data\source\Controle de Ações e FII*.xlsx`

**Destino:**
- Caminho Delta: `D:\Projetos\Jornada_financas_pessoais\data\delta\bronze\`
- Tabela Delta: `bronze.raw_controleativo`


## Imports

In [1]:
from delta import configure_spark_with_delta_pip
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pandas as pd
import glob
import os

## Start Spark Session

In [2]:
# Inicializa uma SparkSession com metastore persistente
builder = (
    SparkSession.builder
    .appName("Leitura de arquivo de formato excel")
    .master("local[*]")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.sql.warehouse.dir", "D:/Projetos/DataLake/spark-warehouse")
    .config(
        "javax.jdo.option.ConnectionURL",
        "jdbc:derby:;databaseName=D:/Projetos/DataLake/metastore_db;create=true"
    )
    .enableHiveSupport()
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

print(f"\n✅ Spark {spark.version} iniciado com metastore persistente!\n")


✅ Spark 3.5.7 iniciado com metastore persistente!



## Variables

In [3]:
# Define o caminho do diretório dos arquivos de origem
source_path = "D:/Projetos/Jornada_financas_pessoais/data/source"

# Define caminhos locais onde serão armazenadas as tabelas Delta
base_bronze_path = "D:/Projetos/Jornada_financas_pessoais/data/delta/bronze"

# Define o caminho da tabela Delta Source
delta_path_controleativo = f"{base_bronze_path}/raw_controleativo"

## Read Excel Files

In [4]:
# Lista todos os arquivos Excel que começam com "Controle de Ações e FII"
file_list = glob.glob(f'{source_path}/Controle de Ações e FII*.xlsx')

print(f"Arquivos Excel encontrados: {len(file_list)}")
for f in file_list:
    print(f"  - {os.path.basename(f)}")

# Lê todos os arquivos Excel e adiciona coluna com nome do arquivo
dfs_list = []

for file_path in file_list:
    print(f"\nProcessando: {os.path.basename(file_path)}")
    
    # Lê apenas a aba "FII"
    # header=0 (padrão) usa a primeira linha como nome das colunas
    df_temp = pd.read_excel(
        file_path,
        sheet_name='FII',
        engine='openpyxl',
        header=0,  # Primeira linha (índice 0) = cabeçalho (PADRÃO)
        dtype=str  # Força todas as colunas como string
    )
    
    # Faz o rename das colunas para casar com a tabela Delta
    df_temp.rename(columns={
        "DATA": "data",
        "PAPEL": "papel",
        "OP": "op",
        "QTD": "qtd",
        "PREÇO": "preco",
        "CUS.T.": "custo_total",
        "TOTAL OP": "total_op",
        "RAT.": "rateio",
        "V. LIQUIDO": "v_liquido",
        "ESTOQUE": "estoque",
        "PMEDIO": "pmedio",
        "GAN/PER": "gan_per",
        "IR MÊS": "ir_mes",
        "MÊS REF.": "mes_ref",
        "VENDAS/MÊS": "vendas_mes",
        "CPF": "cpf",
        "COTISTA": "cotista"
    }, inplace=True)

    # Adiciona coluna com nome do arquivo (apenas o nome, sem path)
    df_temp['arquivo_origem'] = os.path.basename(file_path)
    
    dfs_list.append(df_temp)
    print(f"  ✓ {len(df_temp)} registros lidos")

Arquivos Excel encontrados: 1
  - Controle de Ações e FII - Taynara.xlsx

Processando: Controle de Ações e FII - Taynara.xlsx
  ✓ 63 registros lidos


## Transformation

In [5]:
# Concatena todos os DataFrames em um só
df_controleativo_pd = pd.concat(dfs_list, ignore_index=True)

# Converte o DataFrame pandas para Spark
df_controleativo = spark.createDataFrame(df_controleativo_pd)

# Força todas as colunas para STRING
df_controleativo = df_controleativo.select(
    [col(c).cast("string").alias(c) for c in df_controleativo.columns]
)

print(df_controleativo.count(), "registros lidos")

63 registros lidos


## Save Data Frame

In [6]:
# Grava em formato Delta
(
    df_controleativo.write.format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "false")
    .save(delta_path_controleativo)
)

print(f"[SUCESSO] DataFrame gravado no Delta Lake em: {delta_path_controleativo}")

[SUCESSO] DataFrame gravado no Delta Lake em: D:/Projetos/Jornada_financas_pessoais/data/delta/bronze/raw_controleativo


## Stop Spark Session

In [7]:
# Encerra a SparkSession
spark.stop()